# Importing Libaries

In [20]:
import matplotlib.pyplot as plt
import itertools
import json
import numpy as np
from pylhe import read_lhe_with_attributes, read_num_events
import itertools
import json
import math
import vector
import mplhep as hep
from scipy.optimize import curve_fit

# Defining Functions

## Importing Data Functions

In [21]:
def extract_all_particle_data(lhe_file):
    events = read_lhe_with_attributes(lhe_file)
    num_events = read_num_events(lhe_file)

    particle_data = []

    for event_index, event in enumerate(itertools.islice(events, num_events)):
        for particle in event.particles:
            particle_data.append({
                'event': event_index,
                'id': particle.id,
                'px': particle.px,
                'py': particle.py,
                'pz': particle.pz,
                'E': particle.e,
                'status': particle.status,
                'helicity': getattr(particle, 'spin', None)
            })

    return particle_data

def extract_metadata_from_lhe(lhe_file):
    cross_section = None
    com_energy = None
    num_events = None

    with open(lhe_file, 'r') as file:
        for line in file:
            if "Number of Events" in line:
                num_events = int(line.split(":")[1].strip())

            if "Integrated weight (pb)" in line:
                cross_section = float(line.split(":")[1].strip())

            if "<init>" in line:
                init_line = next(file).strip().split()
                beam_energy_1 = float(init_line[2])
                beam_energy_2 = float(init_line[3])
                com_energy = (beam_energy_1 + beam_energy_2) / 1000  # TeV
                break

    return {"cross_section": cross_section, "center_of_mass_energy": com_energy, "number_of_events": num_events}


def filter_by_id(particle_data, particle_id):
    return [particle for particle in particle_data if particle['id'] == particle_id]

### Constructing Momentums 

In [22]:
def construct_four_vector(data_set):

    four_vector = vector.array({"E": [data["E"] for data in data_set], 
                                "px": [data["px"] for data in data_set], 
                                "py": [data["py"] for data in data_set], 
                                "pz": [data["pz"] for data in data_set]})
    
    return four_vector

def construct_three_momentum(four_vector):

    three_momentum = vector.array({"px": four_vector.px, 
                                   "py": four_vector.py, 
                                   "pz": four_vector.pz})
    return three_momentum

## Validation Functions

### Invariant Mass Check

In [23]:
def invariant_mass_check(vector_1, vector_2):
    inv_masss_1 = vector_1.E**2 - ( vector_1.px**2 + vector_1.py**2 + vector_1.pz**2 )
    inv_masss_2 = vector_2.E**2 - ( vector_2.px**2 + vector_2.py**2 + vector_2.pz**2 )

    mass_diff = np.isclose(inv_masss_1, inv_masss_2, atol=1e-7)
    
    if np.all(mass_diff):
        return

    else:
        changed_indices = np.where(~mass_diff)[0]
        for idx in changed_indices:
            print(f"Warning: Invariant mass changed at index {idx}! Initial: {vector_1.mass[idx]}, Final: {vector_2.mass[idx]}")
    
    return

### Momentum Conservation Check

In [24]:
def momentum_conservation_check(muon_vector, anti_muon_vector):

    total_px = muon_vector.px + anti_muon_vector.px
    total_py = muon_vector.py + anti_muon_vector.py
    total_pz = muon_vector.pz + anti_muon_vector.pz

    px_check = np.isclose(total_px, 0, atol=1e-5)
    py_check = np.isclose(total_py, 0, atol=1e-5)
    pz_check = np.isclose(total_pz, 0, atol=1e-5)

    # Check for px
    if np.all(px_check):
        return
    else:
        failed_indices = np.where(~px_check)[0]
        for idx in failed_indices:
            print(f"Warning: Total px failed at index {idx}: Value = {total_px[idx]}")

    # Check for py
    if np.all(py_check):
        return
    else:
        failed_indices = np.where(~py_check)[0]
        for idx in failed_indices:
            print(f"Warning: Total py failed at index {idx}: Value = {total_py[idx]}")

    # Check for pz
    if np.all(pz_check):
        return
    else:
        failed_indices = np.where(~pz_check)[0]
        for idx in failed_indices:
            print(f"Warning: Total pz failed at index {idx}: Value = {total_pz[idx]}")

    # Final summary
    overall_passed = np.all(px_check) and np.all(py_check) and np.all(pz_check)
    if overall_passed:
        return
    else:
        print("Momentum conservation check failed.")

    return overall_passed

### Rotation Check

In [25]:
def rotation_check(original_vector, rotated_vector, rotation_axis, tolerance=1e-6):


    # Normalize the rotation axis
    rotation_axis_norm = abs(rotation_axis)
    if rotation_axis_norm < tolerance:
        print("The rotation axis is too small or zero.")
        return False

    rotation_axis_normalized = [
        rotation_axis.px / rotation_axis_norm,
        rotation_axis.py / rotation_axis_norm,
        rotation_axis.pz / rotation_axis_norm
    ]

    # Normalize original and rotated vectors
    original_norm = abs(original_vector)
    rotated_norm = abs(rotated_vector)

    # Check if both vectors are non-zero
    if np.all(original_norm < tolerance) or np.all(rotated_norm < tolerance):
        print("One of the vectors is too small or zero.")
        return False

    # Get the angle between the original and rotated vectors using the dot product
    dot_product = (rotation_axis.px * rotated_vector.px +
                   rotation_axis.py * rotated_vector.py +
                   rotation_axis.pz * rotated_vector.pz)

    angle = np.arccos(dot_product / (rotation_axis_norm * rotated_norm))
    angle_deg = np.degrees(angle)

    # Check the angle to determine if the rotation is correct
    # The expected angle depends on your application; you can set it or calculate it based on the specific rotation
    expected_angle_1 = np.full(angle_deg.shape, 0)  # Same size as angle_deg
    expected_angle_2 = np.full(angle_deg.shape, 180.0)  # Same size as angle_deg
    
    angle_diff_1 = np.isclose(angle_deg, expected_angle_1, atol=tolerance)
    angle_diff_2 = np.isclose(angle_deg, expected_angle_2, atol=tolerance)
    combined_angle_diff = np.logical_or(angle_diff_1, angle_diff_2)


    if np.all(combined_angle_diff):
        return

    else:
        changed_indices = np.where(~combined_angle_diff)[0]
        for idx in changed_indices:
            print(f"Warning: Rotation check failed at Index {idx}."
              f"Calculated angle: {angle_deg[idx]} degrees.")




## Rotation Function

In [26]:
def rotate_vector(affected_vector, rotation_axes, rotation_angles):

    rotated_x = np.array(
        (np.cos(rotation_angles) + (1 - np.cos(rotation_angles)) * rotation_axes.x**2) * affected_vector.x +
        (rotation_axes.x * rotation_axes.y * (1 - np.cos(rotation_angles))) * affected_vector.y +
        rotation_axes.y * np.sin(rotation_angles) * affected_vector.z
    )
    
    rotated_y = np.array(
        rotation_axes.px * rotation_axes.py * (1 - np.cos(rotation_angles)) * affected_vector.px +
        (np.cos(rotation_angles) + (1 - np.cos(rotation_angles)) * rotation_axes.py**2) * affected_vector.py -
        rotation_axes.px * np.sin(rotation_angles) * affected_vector.z
    )
        
    rotated_z = np.array(
        -rotation_axes.py * np.sin(rotation_angles) * affected_vector.px +
        rotation_axes.px * np.sin(rotation_angles) * affected_vector.py +
        np.cos(rotation_angles) * affected_vector.z
    )

    rotated_vector = vector.array({"E": affected_vector.E, "px": rotated_x, "py": rotated_y,"pz": rotated_z}) 

    return rotated_vector



## Boost Function

In [27]:
def boost_vector(affected_vector, reference_vector):
    
    beta = reference_vector.pz / reference_vector.E
    gamma = 1 / np.sqrt(1 - beta**2)

    E_boosted = gamma * (affected_vector.E - beta*affected_vector.pz)
    pz_boosted = gamma * (affected_vector.pz - beta * affected_vector.E )
    px_boosted = affected_vector.px
    py_boosted = affected_vector.py

    boosted_vector = vector.array({"E": E_boosted, "px": px_boosted, "py": py_boosted,"pz": pz_boosted}) 

    return boosted_vector

## Rotate and Boost

In [28]:
def rotate_and_boost(affected_vector, reference_vector, axis):

    # Define Three Vectors
    affected_three_vector = construct_three_momentum(affected_vector)
    reference_three_vector = construct_three_momentum(reference_vector)

    # Finding Rotaton Axes and Angles
    rotation_axes = reference_three_vector.cross(axis).unit()
    cos_theta = reference_three_vector.dot(axis) / abs(reference_three_vector) * abs(axis)
    rotation_angles = np.arccos(cos_theta)

    # Rotate reference vector
    rotated_reference_four_vector = rotate_vector(reference_vector, rotation_axes, rotation_angles)
    rotated_reference_three_vector = vector.array({"px": rotated_reference_four_vector.px, "py": rotated_reference_four_vector.py,"pz": rotated_reference_four_vector.pz}) 
    invariant_mass_check(reference_vector, rotated_reference_four_vector)
    rotation_check(reference_three_vector, rotated_reference_three_vector, axis)

    # Rotate affected vector
    rotated_affected_four_vector = rotate_vector(affected_vector, rotation_axes, rotation_angles)
    rotated_affected_three_vector = vector.array({"px": rotated_affected_four_vector.px, "py": rotated_affected_four_vector.py,"pz": rotated_affected_four_vector.pz}) 
    invariant_mass_check(affected_vector, rotated_affected_four_vector)

    # Boost the affected vector
    rotated_boosted_affected_vector = boost_vector(rotated_affected_four_vector, rotated_reference_four_vector )
    invariant_mass_check(rotated_affected_four_vector, rotated_boosted_affected_vector)

    return rotated_boosted_affected_vector

## Angle Functions

In [29]:
def find_polar_angle(four_vector):
    cos_theta =  four_vector.pz / np.sqrt(four_vector.px**2 + four_vector.py**2 + four_vector.pz**2)
    theta = np.arccos(cos_theta)
    return theta

def find_azimuthal_angle(four_vector):
    phi = np.arctan2(four_vector.py, four_vector.px)
    return phi


## Helicity Functions

In [30]:
def z_helicity_function(cos_theta, fL, fR, f0):
    cv = -1/2 + 2* np.sin(0.23152)**2
    ca = -1/2
    alpha = (2*cv*ca)/ (cv**2+ca**2)
    """The fitting function with the provided equation."""
    term1 = (3/8) * fL * (1 + 2 * alpha * cos_theta + cos_theta**2)
    term2 = (3/8) * fR * (1 + cos_theta**2 - 2 * alpha * cos_theta)
    term3 = (3/4) * f0 * (1 - cos_theta**2)
    return term1 + term2 + term3

def w_helicity_function(cos_theta, fL, fR, f0):
    term1 = (3/8) * fL * (1 - cos_theta)**2
    term2 = (3/8) * fR * (1 + cos_theta)**2
    term3 = (3/4) * f0 * (1-cos_theta**2)  
    return term1 + term2 + term3

def anti_w_helicity_function(cos_theta, fL, fR, f0):
    term1 = (3/8) * fL * (1 + cos_theta)**2
    term2 = (3/8) * fR * (1 - cos_theta)**2
    term3 = (3/4) * f0 * (1-cos_theta**2)  
    return term1 + term2 + term3

In [31]:
def calculate_reduced_chi_squared(observed_values, expected_values, error, degrees_of_freedom):
    chi_squared = np.sum(((observed_values - expected_values) ** 2) / error**2)
    reduced_chi_squared = chi_squared / degrees_of_freedom
    return reduced_chi_squared

In [32]:
def calculate_helicity_fractions(data, boson_type, cross_section, num_events, com_energy):

    luminosity = num_events / cross_section

    # Filter out NaN and infinite values
    valid_data = data[np.isfinite(data)]
    cos_theta_data = np.cos(valid_data)
    cos_theta_bins = np.linspace(-1, 1, 50)

    # Set ATLAS style if mplhep is available
    plt.style.use(hep.style.ATLAS)

    fig, ax = plt.subplots(figsize=(8, 6), dpi=100)

    # Calculate histogram values without weights, for accurate error calculation
    raw_counts, bins = np.histogram(cos_theta_data, bins=cos_theta_bins)
    bin_centers = 0.5 * (bins[:-1] + bins[1:])  # Calculate bin centers

    # Calculate errors before applying weights
    errors = np.sqrt(raw_counts)  # Standard error for each bin (sqrt of count)

    # Define weights per event for differential cross-section normalization
    weight_per_event = 1 / (luminosity * (cos_theta_bins[1] - cos_theta_bins[0]) * cross_section)

    # Apply weights to histogram counts and errors
    hist_values = raw_counts * weight_per_event
    errors = errors * weight_per_event

    # Plot the histogram as a line plot with steps and add error bars
    ax.plot(bin_centers, hist_values, drawstyle='steps-mid', color='blue', linewidth=1.5,
            label=r'$\frac{d\sigma}{d(\cos \theta)}$')
    ax.errorbar(bin_centers, hist_values, yerr=errors, fmt='o', color='green', capsize=3, markersize=4)

    # Select helicity function based on boson type
    if boson_type == "Z":
        helicity_function = z_helicity_function
    elif boson_type == "W+":
        helicity_function = w_helicity_function
    elif boson_type == "W-":
        helicity_function = anti_w_helicity_function

    # Fitting
    initial_guess = [0.5, 0.1, 0.1]
    bounds = (0, 1)

    popt, pcov = curve_fit(helicity_function, bin_centers, hist_values, sigma=errors, p0=initial_guess, bounds=bounds)
    total = sum(popt)
    normalized_popt = popt / total if total > 0 else popt 

    # Calculate uncertainties for each helicity fraction
    uncertainties = np.sqrt(np.diag(pcov)) / total if total > 0 else np.sqrt(np.diag(pcov))

    dof = len(hist_values) - len(popt)
    reduced_chi_squared = calculate_reduced_chi_squared(hist_values, helicity_function(bin_centers, *popt), errors, dof)

    # Plot fitted curve
    ax.plot(bin_centers, helicity_function(bin_centers, *popt), color='red', label='Fit', linewidth=2)
    
    # Annotate with helicity fractions, uncertainties, and reduced chi-squared
    ax.text(0.5, 0.9, f"$f_L$ = {normalized_popt[0]:.3f} ± {uncertainties[0]:.3f}", transform=ax.transAxes, fontsize=12, ha='center')
    ax.text(0.5, 0.85, f"$f_R$ = {normalized_popt[1]:.3f} ± {uncertainties[1]:.3f}", transform=ax.transAxes, fontsize=12, ha='center')
    ax.text(0.5, 0.8, f"$f_0$ = {normalized_popt[2]:.3f} ± {uncertainties[2]:.3f}", transform=ax.transAxes, fontsize=12, ha='center')
    ax.text(0.5, 0.75, f"$\chi^2_{{r}}$ = {reduced_chi_squared:.3f}", transform=ax.transAxes, fontsize=12, ha='center')

    ax.set_xlabel(r'cos($\theta^*$)', fontsize=14)
    ax.set_ylabel(r'$\frac{1}{\sigma}\frac{d\sigma}{d(\cos \theta^*)}$', fontsize=16)
    ax.set_title(f"Helicity Fit {boson_type} Boson", fontsize=16, pad=20)
    ax.legend(loc='upper right')
    ax.grid(True, linestyle='--', linewidth=0.5, alpha=0.7)

    # Adjust layout
    plt.tight_layout()

    # Define save path relative to current directory, with sanitized boson type
    save_dir = f"./Plots/{com_energy}TeV/Helicity Fractions"
    os.makedirs(save_dir, exist_ok=True)  # Create the directory if it doesn't exist
    sanitized_boson_type = boson_type.replace("+", "plus").replace("-", "minus")  # Remove symbols for filename
    save_path = os.path.join(save_dir, f"Helicity_Distribution_{sanitized_boson_type}_{com_energy}TeV.png")

    # Save the plot
    plt.savefig(save_path, dpi=300)
    plt.show()

    return normalized_popt, uncertainties


## Plotting Functions


In [33]:
import os
import numpy as np
import matplotlib.pyplot as plt

def plot_cos_distribution(theta_data, cross_section, com_energy, num_events, decay_product=""):
    """
    Plots theta and cos(theta) distributions with error bars and saves to a local file path.
    
    Parameters:
    - theta_data: Array of theta angles in radians.
    - cross_section: Cross section of the process (e.g., in fb or pb).
    - com_energy: Center of mass energy (e.g., in TeV).
    - num_events: Total number of events, used for weighting the histogram.
    - decay_product: String representing the decay product (e.g., "mu" for muon), used as a subscript in labels.
    """
    # Calculate luminosity
    luminosity = num_events / cross_section
    
    # Compute cos(theta) for the second histogram
    cos_theta_data = np.cos(theta_data)
    
    # Set up the figure
    plt.figure(figsize=(12, 6))
    
    # Plot for theta with specific weights and error bars
    plt.subplot(1, 2, 1)
    raw_counts_theta, bin_edges_theta = np.histogram(theta_data, bins=100, range=(0, np.pi))
    bin_centers_theta = 0.5 * (bin_edges_theta[1:] + bin_edges_theta[:-1])
    bin_width_theta = bin_edges_theta[1] - bin_edges_theta[0]         

    # Define weight per event for the theta plot
    weight_per_event_theta = 1 / (luminosity * bin_width_theta * cross_section)

    # Calculate weighted counts and errors for theta
    counts_theta = raw_counts_theta * weight_per_event_theta
    errors_theta = np.sqrt(raw_counts_theta) * weight_per_event_theta  # Weighted errors

    # Plot with weighted counts and error bars for theta
    plt.plot(bin_centers_theta, counts_theta, drawstyle='steps-mid', color='blue', linewidth=1.5)
    plt.errorbar(bin_centers_theta, counts_theta, yerr=errors_theta, fmt='o', color='blue', capsize=3, markersize=4)
    plt.xlabel(rf'$\theta_{{{decay_product}}}$ (radians)')
    plt.ylabel(rf'$\frac{{d\sigma}}{{d\theta_{{{decay_product}}}}}$')
    plt.title(rf'Angular Distribution of $\theta_{{{decay_product}}}$', fontsize=14)
    
    # Plot for cos(theta) with specific weights and error bars
    plt.subplot(1, 2, 2)
    raw_counts_cos_theta, bin_edges_cos_theta = np.histogram(cos_theta_data, bins=100, range=(-1, 1))
    bin_centers_cos_theta = 0.5 * (bin_edges_cos_theta[1:] + bin_edges_cos_theta[:-1])  # Calculate bin centers
    bin_width_cos_theta = bin_edges_cos_theta[1] - bin_edges_cos_theta[0]               # Bin width for cos(theta)

    # Define weight per event for the cos(theta) plot
    weight_per_event_cos_theta = 1 / (luminosity * bin_width_cos_theta * cross_section)

    # Calculate weighted counts and errors for cos(theta)
    counts_cos_theta = raw_counts_cos_theta * weight_per_event_cos_theta
    errors_cos_theta = np.sqrt(raw_counts_cos_theta) * weight_per_event_cos_theta  # Weighted errors

    # Plot with weighted counts and error bars for cos(theta)
    plt.plot(bin_centers_cos_theta, counts_cos_theta, drawstyle='steps-mid', color='red', linewidth=1.5)
    plt.errorbar(bin_centers_cos_theta, counts_cos_theta, yerr=errors_cos_theta, fmt='o', color='red', capsize=3, markersize=4)
    plt.xlabel(rf'$\cos(\theta_{{{decay_product}}})$')
    plt.ylabel(rf'$\frac{{d\sigma}}{{d\cos(\theta_{{{decay_product}}})}}$')
    plt.title(rf'Angular Distribution of $\cos(\theta_{{{decay_product}}})$', fontsize=14)
    
    # Add text for cross-section and center of mass energy
    plt.suptitle(f'Angular Distributions\nCross Section: {cross_section} fb, Center of Mass Energy: {com_energy} TeV',
                 fontsize=14, fontweight='bold')
    
    # Adjust layout
    plt.tight_layout(rect=[0, 0, 1, 0.95])

    # Define save path relative to current directory, replacing any characters that may interfere with the path
    save_dir = f"./Plots/{com_energy}TeV/Theta Distribution"
    os.makedirs(save_dir, exist_ok=True)  # Create the directory if it doesn't exist
    sanitized_decay_product = decay_product.replace("\\", "").replace("^", "")  # Remove LaTeX characters for filename
    save_path = os.path.join(save_dir, f"Theta_Distribution_{sanitized_decay_product}_{com_energy}TeV.png")

    # Save the plot
    plt.savefig(save_path, dpi=300)
    plt.show()


In [34]:
import os
import numpy as np
import matplotlib.pyplot as plt

def plot_phi_distribution(phi_data, cross_section, com_energy, num_events, decay_product=""):
    """
    Plots the angular distribution for phi with error bars and a residuals plot, saving the plot locally.
    
    Parameters:
    - phi_data: Array of phi angles in radians.
    - cross_section: Cross section of the process (e.g., in fb or pb).
    - com_energy: Center of mass energy (e.g., in TeV).
    - num_events: Total number of events, used for weighting the histogram.
    - decay_product: String representing the decay product (e.g., "mu" for muon), used as a subscript in labels.
    """
    # Calculate luminosity
    luminosity = num_events / cross_section
    
    # Set up the figure with main plot and residuals plot
    fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(8, 8), gridspec_kw={'height_ratios': [3, 1]}, sharex=True)
    
    # Histogram for phi with specific weights and error bars
    raw_counts_phi, bin_edges_phi = np.histogram(phi_data, bins=100, range=(-np.pi, np.pi))
    bin_centers_phi = 0.5 * (bin_edges_phi[1:] + bin_edges_phi[:-1])  # Calculate bin centers
    bin_width_phi = bin_edges_phi[1] - bin_edges_phi[0]               # Calculate bin width
    
    # Define weight per event for the phi plot
    weight_per_event_phi = 1 / (luminosity * bin_width_phi * cross_section)
    
    # Calculate weighted counts and errors for phi
    counts_phi = raw_counts_phi * weight_per_event_phi
    errors_phi = np.sqrt(raw_counts_phi) * weight_per_event_phi  # Weighted errors
    
    # Plot with weighted counts and error bars for phi
    ax1.plot(bin_centers_phi, counts_phi, drawstyle='steps-mid', color='purple', linewidth=1.5)
    ax1.errorbar(bin_centers_phi, counts_phi, yerr=errors_phi, fmt='o', color='purple', capsize=3, markersize=4)
    ax1.set_ylabel(rf'$\frac{{d\sigma}}{{d\phi_{{{decay_product}}}}}$')
    ax1.set_title(rf'Angular Distribution of $\phi_{{{decay_product}}}$', fontsize=14)
    
    # Reference line (mean of the weighted counts)
    mean_value = np.mean(counts_phi)
    ax1.axhline(mean_value, color='gray', linestyle='--', label='Mean Value')
    ax1.legend()
    
    # Calculate residuals as (observed - mean)
    residuals_phi = counts_phi - mean_value
    
    # Plot residuals with error bars
    ax2.errorbar(bin_centers_phi, residuals_phi, yerr=errors_phi, fmt='o', color='purple', capsize=3, markersize=4)
    ax2.axhline(0, color='black', linestyle='--')
    ax2.set_xlabel(rf'$\phi_{{{decay_product}}}$ (radians)')
    ax2.set_ylabel('Residuals')

    # Add text for cross-section and center of mass energy
    fig.suptitle(f'Phi Angular Distribution with Residuals\nCross Section: {cross_section} fb, Center of Mass Energy: {com_energy} TeV',
                 fontsize=14, fontweight='bold')
    
    # Adjust layout
    plt.tight_layout(rect=[0, 0, 1, 0.95])

    # Define save path relative to the current directory, with a sanitized decay product
    save_dir = f"./Plots/{com_energy}TeV/Phi Distribution"
    os.makedirs(save_dir, exist_ok=True)  # Create the directory if it doesn't exist
    sanitized_decay_product = decay_product.replace("\\", "").replace("^", "")  # Remove LaTeX characters for filename
    save_path = os.path.join(save_dir, f"Phi_Distribution_{sanitized_decay_product}_{com_energy}TeV.png")

    # Save the plot
    plt.savefig(save_path, dpi=300)
    plt.show()


In [35]:
def apply_boosts(four_momenta, com_momentum, z_axis, transform_func):
    boosted_momenta = {}
    for name, momentum in four_momenta.items():
        boosted_momenta[name] = transform_func(momentum, com_momentum, z_axis)
    return boosted_momenta

# Main Code

### Importing Data 

In [ ]:
import numpy as np
import vector  # Assuming vector is being used for four-vector operations

# Define your particle IDs for easy reuse
particle_ids = {
    "z": 23, "muon": 13, "anti_muon": -13, "w": 24, "electron_neutrino": 12, 
    "anti_electron": -11, "w_anti": -24, "electron": 11, "anti_electron_neutrino": -12
}

def process_energy(lhe_file):
    # Extract particle data and metadata
    all_particle_data = extract_all_particle_data(lhe_file)
    metadata = extract_metadata_from_lhe(lhe_file)
    
    CrossSection = metadata["cross_section"]
    CentreOfMassEnergy = metadata["center_of_mass_energy"]
    NumberOfEvents = metadata["number_of_events"]

    # Filter and organize data by particle
    particle_data = {name: filter_by_id(all_particle_data, pid) for name, pid in particle_ids.items()}
    four_momenta = {name: construct_four_vector(data) for name, data in particle_data.items()}
    com_four_momentum = four_momenta["z"] + four_momenta["w"] + four_momenta["w_anti"]
    z_axis = vector.obj(px=0, py=0, pz=1)
    
    # Apply transformations
    boosted_momenta_1 = apply_boosts(four_momenta, com_four_momentum, z_axis, rotate_and_boost)
    
    # Z boson decay products
    muon_rest = rotate_and_boost(boosted_momenta_1["muon"], boosted_momenta_1["z"], z_axis)
    anti_muon_rest = rotate_and_boost(boosted_momenta_1["anti_muon"], boosted_momenta_1["z"], z_axis)
    
    # W boson decay products
    electron_neutrino_rest = rotate_and_boost(boosted_momenta_1["electron_neutrino"], boosted_momenta_1["w"], z_axis)
    anti_electron_rest = rotate_and_boost(boosted_momenta_1["anti_electron"], boosted_momenta_1["w"], z_axis)
    
    # Anti-W boson decay products
    electron_rest = rotate_and_boost(boosted_momenta_1["electron"], boosted_momenta_1["w_anti"], z_axis)
    anti_electron_neutrino_rest = rotate_and_boost(boosted_momenta_1["anti_electron_neutrino"], boosted_momenta_1["w_anti"], z_axis)

    # Angle calculations
    angles = {
        "muon_polar": find_polar_angle(muon_rest),
        "anti_muon_polar": find_polar_angle(anti_muon_rest),
        "electron_neutrino_polar": find_polar_angle(electron_neutrino_rest),
        "anti_electron_polar": find_polar_angle(anti_electron_rest),
        "electron_polar": find_polar_angle(electron_rest),
        "anti_electron_neutrino_polar": find_polar_angle(anti_electron_neutrino_rest),
        "muon_azimuthal": find_azimuthal_angle(muon_rest),
        "anti_muon_azimuthal": find_azimuthal_angle(anti_muon_rest),
        "electron_neutrino_azimuthal": find_azimuthal_angle(electron_neutrino_rest),
        "anti_electron_azimuthal": find_azimuthal_angle(anti_electron_rest),
        "electron_azimuthal": find_azimuthal_angle(electron_rest),
        "anti_electron_neutrino_azimuthal": find_azimuthal_angle(anti_electron_neutrino_rest)
    }

    # Perform momentum conservation checks
    momentum_checks = {
        "z_decay": momentum_conservation_check(muon_rest, anti_muon_rest),
        "w_decay": momentum_conservation_check(electron_neutrino_rest, anti_electron_rest),
        "w_anti_decay": momentum_conservation_check(electron_rest, anti_electron_neutrino_rest)
    }

    return {
        "metadata": metadata,
        "four_momenta": four_momenta,
        "angles": angles,
        "momentum_checks": momentum_checks
    }

def process_multiple_energies(energy_files):
    """
    Processes data for multiple energy levels and stores the results in a dictionary.

    Parameters:
        energy_files (dict): Dictionary where keys are energy levels (e.g., 13.6) and values are LHE file paths.

    Returns:
        dict: Dictionary with results for each energy level.
    """
    results_by_energy = {}
    for energy, lhe_file in energy_files.items():
        print(f"Processing {energy} TeV")
        results_by_energy[energy] = process_energy(lhe_file)
    return results_by_energy

# Usage example
energy_files = {
    13.6:  r"C:\Users\gabri\Documents\University\Year 4\Mphys\DataFiles\wwz_500k_13.6TeV.lhe",
    14.0:  r"C:\Users\gabri\Documents\University\Year 4\Mphys\DataFiles\wwz_500k_14TeV.lhe"
}

results = process_multiple_energies(energy_files)


Processing 13.6 TeV
Processing 14.0 TeV
